In [1]:
!pip install boto3

  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)


You should consider upgrading via the 'F:\Git\info-panel\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [13]:
!pip install pyperclip

Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
    Running setup.py install for pyperclip: started
    Running setup.py install for pyperclip: finished with status 'done'


You should consider upgrading via the 'F:\Git\info-panel\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import boto3
import pyperclip

In [9]:
final_instances = list()

In [15]:
def retrieve_ec2_instances():
    ec2 = boto3.resource('ec2')
    instances = ec2.instances.all()

    print(f'{"ID":<2} {"Instance ID":<20} {"Name":<20} {"State":<15} {"Instance Type":<15} {"Public IPv4":<20}')
    final_instances.clear()
    for key, instance in enumerate(instances):
        name = ''
        for tag in instance.tags or []:
            if tag['Key'] == 'Name':
                name = tag['Value']
        final_instances.append((instance.id, name, instance.public_ip_address or ""))
        print(f'{key:<2} {instance.id:<20} {name:<20} {instance.state["Name"]:<15} {instance.instance_type:<15} {instance.public_ip_address or "N/A":<20}')


In [44]:
retrieve_ec2_instances()

ID Instance ID          Name                 State           Instance Type   Public IPv4         
0  i-0bf573cc6b5702a0a  Nodejs App Testing   stopped         t2.micro        N/A                 
1  i-0218c10256d6d7548  Sona Question Bank Preview stopped         t2.micro        N/A                 
2  i-0a536c0985169f16b  Proxy Tunnel         running         t2.micro        34.201.133.110      
3  i-0046d8799738f6c74  CLI Testing          terminated      t2.micro        N/A                 
4  i-02932c7effc3d8797  CLI Testing          terminated      t2.micro        N/A                 
5  i-0ef8b70a295e95b5b  CLI Testing          terminated      t2.micro        N/A                 
6  i-0388fe17fe4c1e1ca  Info Panel Testing 5 running         t2.micro        54.175.230.49       


In [13]:
def create_ec2_instance():
    ec2 = boto3.resource('ec2')
    
    # Gather user inputs
    instance_name = input("Enter Instance Name: ")
    instance_type = input("Enter Instance Type (default: t2.micro): ") or 't2.micro'
    
    # Fetch available key pairs
    key_pairs_client = boto3.client('ec2')
    key_pairs_response = key_pairs_client.describe_key_pairs()
    key_pairs = key_pairs_response['KeyPairs']
    
    print("Key Pairs Available:")
    for i, key_pair in enumerate(key_pairs):
        print(f"{i}. {key_pair['KeyName']}")
        
    key_pair_index = int(input("Enter your Key Pair (index): "))
    key_name = key_pairs[key_pair_index]['KeyName']  # Get selected key pair name
    
    # Firewall selection (Security Groups)
    security_groups = [
        ('Production Website Group', 'sg-090235eb6bc9b2a0e'),
        ('Allow All', 'sg-04496392c2efa7f7f'),
    ]
    
    print("Select Firewall:")
    for index, (name, sg_id) in enumerate(security_groups):
        print(f"{index}. {name} - {sg_id}")
    
    security_group_index = int(input("Enter your Firewall selection (index): "))
    security_group_ids = security_groups[security_group_index][1]
    storage_size = input("Storage Size (default 8GB): ") or 8  # Default to 8GB
    no_of_instances = input("No Of Instances (default 1): ") or 1  # Default to 1

    # Create the EC2 instance
    instance = ec2.create_instances(
        ImageId='ami-0866a3c8686eaeeba',  # Change this to your desired AMI ID
        MinCount=int(no_of_instances),
        MaxCount=int(no_of_instances),
        InstanceType=instance_type,
        KeyName=key_name,
        SecurityGroupIds=[security_group_ids],
        BlockDeviceMappings=[{
            'DeviceName': '/dev/sda1',  # Modify as needed
            'Ebs': {
                'VolumeSize': int(storage_size),
                'DeleteOnTermination': True
            }
        }]
    )

    # Tag the instance with the provided instance name
    print(f'{"ID":<2} {"Instance ID":<20} {"Name":<20} {"State":<15} {"Instance Type":<15} {"Public IPv4":<20}')
    for key, inst in enumerate(instance):
        inst.create_tags(Tags=[{'Key': 'Name', 'Value': instance_name}])
        inst.reload()
        public_ipv4 = inst.public_ip_address if inst.public_ip_address else "N/A"
        print(f'{key:<2} {inst.id:<20} {instance_name:<20} {inst.state["Name"]:<15} {inst.instance_type:<15} {public_ipv4:<20}')

In [14]:
create_ec2_instance()

Key Pairs Available:
0. Akil Sadik
1. MSI
Select Firewall:
0. Allow All - sg-04496392c2efa7f7f
1. Production Website Group - sg-090235eb6bc9b2a0e
ID Instance ID          Name                 State           Instance Type   Public IPv4         
1  i-0ef8b70a295e95b5b  CLI Testing          pending         t2.micro        18.209.36.210       
1  i-02932c7effc3d8797  CLI Testing          pending         t2.micro        54.224.202.75       


In [19]:
def copy_ip():
    retrieve_ec2_instances()
    choice = int(input("Enter instance ID to copy its ip address: "))
    ip = final_instances[choice][2]
    pyperclip.copy(ip)
    print(f"\n Address: {ip} copied to clipboard successfully")

In [20]:
copy_ip()

ID Instance ID          Name                 State           Instance Type   Public IPv4         
0  i-0bf573cc6b5702a0a  Nodejs App Testing   stopped         t2.micro        N/A                 
1  i-0218c10256d6d7548  Sona Question Bank Preview stopped         t2.micro        N/A                 
2  i-0a536c0985169f16b  Proxy Tunnel         running         t2.micro        34.201.133.110      
3  i-0046d8799738f6c74  CLI Testing          terminated      t2.micro        N/A                 
4  i-02932c7effc3d8797  CLI Testing          running         t2.micro        54.224.202.75       
5  i-0ef8b70a295e95b5b  CLI Testing          running         t2.micro        18.209.36.210       
6  i-0388fe17fe4c1e1ca  Info Panel Testing 5 running         t2.micro        54.175.230.49       
IPv4 Address: 34.201.133.110 copied to clipboard successfully


In [29]:
def stop_ec2_instance():
    retrieve_ec2_instances()
    instances = [final_instances[i][0] for i in map(int, input("\nEnter the instance ID(s) to Stop: ").replace(',', ' ').split())]
    ec2 = boto3.client('ec2')
    ec2.stop_instances(InstanceIds=instances)
    print(f'\nStopped EC2 Instance(s): {", ".join(instances)}')

In [30]:
stop_ec2_instance()

ID Instance ID          Name                 State           Instance Type   Public IPv4         
0  i-0bf573cc6b5702a0a  Nodejs App Testing   stopped         t2.micro        N/A                 
1  i-0218c10256d6d7548  Sona Question Bank Preview stopped         t2.micro        N/A                 
2  i-0a536c0985169f16b  Proxy Tunnel         running         t2.micro        34.201.133.110      
3  i-0046d8799738f6c74  CLI Testing          terminated      t2.micro        N/A                 
4  i-02932c7effc3d8797  CLI Testing          running         t2.micro        54.224.202.75       
5  i-0ef8b70a295e95b5b  CLI Testing          running         t2.micro        18.209.36.210       
6  i-0388fe17fe4c1e1ca  Info Panel Testing 5 running         t2.micro        54.175.230.49       
Stopped EC2 Instance(s): i-02932c7effc3d8797, i-0ef8b70a295e95b5b


In [33]:
def start_ec2_instance():
    retrieve_ec2_instances()
    instances = [final_instances[i][0] for i in map(int, input("\nEnter the instance ID(s) to Start: ").replace(',', ' ').split())]
    ec2 = boto3.client('ec2')
    ec2.start_instances(InstanceIds=instances)
    print(f'\nStarted EC2 Instance(s): {", ".join(instances)}')

In [34]:
start_ec2_instance()

ID Instance ID          Name                 State           Instance Type   Public IPv4         
0  i-0bf573cc6b5702a0a  Nodejs App Testing   stopped         t2.micro        N/A                 
1  i-0218c10256d6d7548  Sona Question Bank Preview stopped         t2.micro        N/A                 
2  i-0a536c0985169f16b  Proxy Tunnel         running         t2.micro        34.201.133.110      
3  i-0046d8799738f6c74  CLI Testing          terminated      t2.micro        N/A                 
4  i-02932c7effc3d8797  CLI Testing          stopped         t2.micro        N/A                 
5  i-0ef8b70a295e95b5b  CLI Testing          stopped         t2.micro        N/A                 
6  i-0388fe17fe4c1e1ca  Info Panel Testing 5 running         t2.micro        54.175.230.49       

Started EC2 Instance(s): i-02932c7effc3d8797, i-0ef8b70a295e95b5b


In [36]:
def delete_ec2_instance():
    retrieve_ec2_instances()
    instances = [final_instances[i][0] for i in map(int, input("\nEnter the instance ID(s) to Delete: ").replace(',', ' ').split())]
    ec2 = boto3.client('ec2')
    ec2.terminate_instances(InstanceIds=instances)
    print(f'\nDeleted EC2 Instance(s): {", ".join(instances)}')

In [38]:
delete_ec2_instance()

ID Instance ID          Name                 State           Instance Type   Public IPv4         
0  i-0bf573cc6b5702a0a  Nodejs App Testing   stopped         t2.micro        N/A                 
1  i-0218c10256d6d7548  Sona Question Bank Preview stopped         t2.micro        N/A                 
2  i-0a536c0985169f16b  Proxy Tunnel         running         t2.micro        34.201.133.110      
3  i-0046d8799738f6c74  CLI Testing          terminated      t2.micro        N/A                 
4  i-02932c7effc3d8797  CLI Testing          running         t2.micro        98.82.36.197        
5  i-0ef8b70a295e95b5b  CLI Testing          running         t2.micro        18.209.163.110      
6  i-0388fe17fe4c1e1ca  Info Panel Testing 5 running         t2.micro        54.175.230.49       

Deleted EC2 Instance(s): i-02932c7effc3d8797, i-0ef8b70a295e95b5b
